# Pinecone

>[Pinecone](https://docs.pinecone.io/docs/overview) 是一个功能丰富的向量数据库。

本 Notebook 展示了如何使用与 `PineconeVectorStore` 向量数据库相关的功能。

## 设置

要使用 `PineconeVectorStore`，您首先需要安装相应的包，以及本 Notebook 中将要使用的其他包。

In [ ]:
pip install -qU langchain langchain-pinecone langchain-openai

迁移注意事项：如果您正在从 `langchain_community.vectorstores` 迁移 Pinecone 实现，您可能需要在安装依赖于 `pinecone-client` v6 的 `langchain-pinecone` 之前，移除 `pinecone-client` v2 依赖。

### 凭证

创建一个新的 Pinecone 账户，或登录您现有的账户，并创建一个 API 密钥以供在此笔记本中使用。

In [ ]:
import getpass
import os

from pinecone import Pinecone

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

如果您想自动跟踪模型的调用，还可以通过取消注释以下行来设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [5]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## 初始化

在初始化向量存储之前，让我们先连接到 Pinecone 索引。如果名为 `index_name` 的索引不存在，它将被创建。

In [4]:
from pinecone import ServerlessSpec

index_name = "langchain-test-index"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

## 管理向量存储

创建完向量存储后，我们可以通过添加和删除不同的项与其进行交互。

### 向向量存储添加项

我们可以使用 `add_documents` 函数向向量存储添加项。

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

### 从向量存储中删除项目

In [8]:
vector_store.delete(ids=[uuids[-1]])

## 查询向量存储

一旦创建了向量存储并将相关文档添加进去，在运行你的链或代理的过程中，你很可能希望对其进行查询。

### 直接查询

执行简单的相似性搜索可以按以下方式完成：

In [9]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

#### 相似度搜索（含分数）

您也可以使用分数进行搜索：

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

#### 其他搜索方法

本笔记本中未列出更多搜索方法（例如 MMR），要查找所有这些方法，请务必阅读 [API 参考](https://python.langchain.com/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html)。

### 通过转换为检索器进行查询

您还可以将向量存储转换为检索器，以便在链中更轻松地使用。

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

## 用于检索增强一代的使用方法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下章节：

- [教程](/docs/tutorials/)
- [操作指南：使用 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval)

## API 参考

有关所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html